In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src', 'mutsuite')))

import configparser
import os
import sys
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from subprocess import call
from collections import defaultdict
from io import StringIO
from mutRun import mutRun
from mutTestHelpers import aggregate_mutations, print_all_mutations_by_sample
from callers.simCaller import SimCaller
from callers.pindelCaller import PindelCaller #additional callers can be implemented and imported here
from callers.mutectCaller import MutectCaller
from callers.lofreqCaller import LofreqCaller
from callers.strelkaCaller import StrelkaCaller
from callers.somaticSniperCaller import SomaticSniperCaller
from callers.vardictCaller import VardictCaller
from callers.varscanCaller import VarscanCaller

from IPython.display import Code

# First, define the Caller class
Individual indel callers can implement this class for easy integration into the testing framework.
As an example, see [pindelCaller.py](pindelCaller.py).

In [3]:
Code(filename='../src/mutsuite/callers/caller.py', language='python')

class Caller:
    def __init__(self):
        """"Instantiate a new caller object.
        Any variables unique to this caller should be set here.
        For example, if this caller requires the path to the fastq reference, that should be set here.
        """
        pass

    def get_name(self):
        """Get the name of the caller.
        This name will be used in output that compares this caller to other callers
        Returns: A string with a name identifying the caller
        """
        pass

    def run_caller(self, sample_bam, control_bam, use_cached=True):
        """Extract indels from a pair of bams.
        sample_bam is the bam with simulated indels
        control_bam is the bam with no simulated indels
        use_cached is a boolean indicating whether to use cached results if they exist. Cached results are accessible even after 'clean' has been called.
        Returns: a tuple of:
            A string with a command to run this indel for this pair of bams
            A boolean for whether the command has already been completed successfully (and should not be run again)
        """
        pass

    def get_results(self, sample_bam, control_bam, use_cached=True):
        """Extract indel calling results.
        sample_bam is the bam with simulated indels
        control_bam is the bam with no simulated indels
        use_cached is a boolean indicating whether to use cached results if they exist. Cached results are accessible even after 'clean' has been called.
        run_caller has been run previously. This function collects the results for indel calling by this caller
        Returns: 
            a dictionary of mutations passing filters: 'LOC TYPE INFO' => count
                where TYPE is either 'I' for insertion, 'D' for deletion, 'S' for substitution or 'M' for mixed/multiple
                if TYPE is 'I' INFO will be the sequence of the insertion
                if TYPE is 'D' INFO will be the length of the deletion
                if TYPE is 'S' INFO will be the substituted base
                e.g. if a 25bp deletion was observed at position 100 supported by 5 reads, the key would be '100 D 25' and the value would be 5
                e.g. if a 2bp insertion of AT was observed at position 50 supported by 3 reads, the key would be '50 I AT' and the value would be 3
            a dictionary of all mutations: 'LOC TYPE INFO' => Mutation object
        """
        pass
    
    def clean(self, sample_bam, control_bam):
        """
        Remove all intermediate files associated with calling mutations for this sample
        """
        pass

## Read in settings file. 
This settings file contains information required to run this simulation, including the location of the genome edits for simulations, and the bam sources of edited and non-edited reads.

In [4]:
settingsFile = "simSettings.txt"
Code(filename=settingsFile)

[Simulation]
depths: 30   # specifies the depths for the simulation, as a comma-separated list
countMut: 5 # specifies the number of reads to be mutated, as a comma separated list
addQual: 0       # specifies the addition of quality to all bases (0 means that original quality scores are used)
reps: 1          # specifies the number of replicates of each depth/pctMut/addQual setting to run
chr: chr11       # location of genome editing event
loc: 836297      # location of genome editing event
FDRtolerance: 10,100,1000 # window sizes for false positive indel calls
reference: genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa        # location of fastq reference sequence, with bwa alignment files
sourceBam: data/NA12878/HG001.GRCh38_full_plus_hs38d1_analysis_set_minus_alts.300x.bam # location of bam with unaltered reads
unalteredBam: 10kbAroundCutsite.BWA.bam # file to write unaltered reads around cut site
unalteredNamesortedBam: 10kbAroundCutsite.BWA.nameSort.bam # file to write unaltered reads around cut site (namesorted)
simulateRange: 5000 # A simulated and control sample will be generated with reads within this many bp of 'chr:'loc' as specified above
onlyInsertSpecificMutation: D1,D2
mutSimScript: ../src/mutsuite/mutSim.py # location of mutSim.py script and module files.

[Callers]
gatk_env_command: conda run -n kc_gatk4 bash -c "
lofreq_env_command: conda run -n kc_lofreq bash -c "
pindel_env_command: conda run -n kc_pindel bash -c "
somaticsniper_env_command: conda run -n kc_somaticSniper bash -c "
strelka_env_command: conda run -n kc_strelka2 bash -c "
vardict_env_command: conda run -n kc_vardict bash -c "
varscan_env_command: conda run -n kc_varscan bash -c "
caller_threads: 20

In [5]:
if not os.path.isfile(settingsFile):
    raise Exception("Couldn't find settings file")
workFolder = os.path.abspath(settingsFile) + ".output/"
if not os.path.isdir(workFolder):
    os.makedirs(workFolder)
Config = configparser.ConfigParser(inline_comment_prefixes="#")
Config.read(settingsFile)

['simSettings.txt']

## Create simulated samples by replacing reads with those with simulated indels

In [6]:
samples, simulateReadsCommands = mutRun(Config, workFolder)
if len(simulateReadsCommands) > 0:
        print ("got " + str(len(simulateReadsCommands)) + " commands to simulate samples")
        for command in simulateReadsCommands:
                print('calling command ' + str(command))
                call(command,shell=True)
else:
        print('Finished simulating samples')

Finished simulating samples


## Read actual simulated reads
We use the [SimCaller](simCaller.py) class, which is a special instance of the Caller class, for which the ```get_results``` method is implemented, and recovers the simulated indels inserted into the simulated samples.

In [7]:
simCaller = SimCaller(Config)
simulated_mutations = {}
for sample_name, simulated_bam, simulated_control in samples:
    sample_simulated_indels, sample_simulated_mutations = simCaller.get_results(simulated_bam, simulated_control)
    simulated_mutations[sample_name] = sample_simulated_mutations
aggregated_simulated_mutations = aggregate_mutations(simulated_mutations)


In [8]:
aggregated_simulated_mutations

defaultdict(int, {'836295 D 1': 5, '836295 D 2': 5})

## Create an array of callers
Users may implement additional callers. They are instantiated here and added to this list of callers for testing and comparison below.

In [9]:
Config = configparser.ConfigParser(inline_comment_prefixes="#")
Config.read(settingsFile)

callers = []
pindelCaller = PindelCaller(Config)
callers.append(pindelCaller)
lofreqCaller = LofreqCaller(Config)
callers.append(lofreqCaller)
vardictCaller = VardictCaller(Config)
callers.append(vardictCaller)
somaticsniperCaller = SomaticSniperCaller(Config)
callers.append(somaticsniperCaller)
varscanCaller = VarscanCaller(Config)
callers.append(varscanCaller)
mutectCaller = MutectCaller(Config)
callers.append(mutectCaller)
strelkaCaller = StrelkaCaller(Config)
callers.append(strelkaCaller)


## Run a single caller
Now that the callers are implemented, we can do a test run on single sample. Here, we'll demonstrate the pindelCaller.

In [10]:
#grab a sample
test_sample_name, test_sample_sim_bam, test_sample_control_bam = samples[-1]
print("Test sample: " + test_sample_name + "\nSimulated bam: " + test_sample_sim_bam + "\nControl bam: " + test_sample_control_bam)


Test sample: sim_d30_p5.0_q0_sD2_r0
Simulated bam: /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.bam
Control bam: /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.ctl.bam


In [11]:
#When we execute run_caller, the caller will return the command to run that caller and a boolean for whether the command has already completed successfully.
command, is_finished = pindelCaller.run_caller(test_sample_sim_bam,test_sample_control_bam)
print('Command is: ' + command)
print("is_finished: " + str(is_finished))

if not is_finished:
    call(command,shell=True) #takes a minute to run

Command is: conda run -n kc_pindel bash -c " pindel -f genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa -r false -t false -l false -k false -s false -i /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.pindel/pindel.config -o /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.pindel/pindel.tmp. -T 20 -c chr11:835197-837397 && pindel2vcf -P /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.pindel/pindel.tmp. -r genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa -R genome -d 20101123 -v /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.pindel/pindel.vcf -

Initializing parameters...
Pindel version 0.2.5b9, 20160729.
Loading reference genome ...
Loading reference genome done.
Initializing parameters done.
SearchRegion::SearchRegion
Processing region: chr11	835197	837397
Chromosome Size: 135086622
NumBoxes: 60008	BoxSize: 4509

Looking at chromosome chr11 bases 835197 to 837397 of the bed region: chromosome chr11:835197-837397 
No discordant RP reads in Bamfile /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.ctl.bam
No discordant RP reads in Bamfile /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.bam
Discovery RP: 0
sorting RP complete.
Reads_RP.size(): 0
Modify RP complete.
adding BD from RP.
modify and summarize interchr RP.
adding BD from interChr RP.
summarize BP as BD complete. Now start sorting BD...
sorting BD... done.
external BD eve

In [12]:
#The get_results command returns a dictionary of 'LOC INDEL INFO' => COUNT for each indel found. 
#For example, the key/value pair {'12345 D 1' : 5}
#   means that a 1bp deletion was found at location 12345 with a coverage of 5 reads.
#We'll print that dictionary of results out here:
test_indels = pindelCaller.get_results(test_sample_sim_bam,test_sample_control_bam)
print(test_indels)

({'835718 D 2': 2, '836294 D 2': 5, '836509 D 1': 2, '836538 D 1': 22}, {'835718 D 2': <mutation.Mutation object at 0x147e55d06d00>, '836294 D 2': <mutation.Mutation object at 0x148202171df0>, '836509 D 1': <mutation.Mutation object at 0x147e558d4b50>, '836538 D 1': <mutation.Mutation object at 0x147e558d4d90>})


[W::bcf_hdr_check_sanity] PL should be declared as Number=G
[W::vcf_parse] Contig 'chr11' is not defined in the header. (Quick workaround: index the file with tabix.)


Now that we have shown the basic functionality of the caller class, we can run all of the callers on each of the simulated samples, and aggregate the results.
## Run each caller on all of the sample pairs

In [13]:
for caller in callers:
    caller_commands = []
    caller_name = caller.get_name()
    for name, simulated_bam, simulated_control in samples:
        this_command, is_finished = caller.run_caller(simulated_bam,simulated_control)
        if not is_finished:
            caller_commands.append(this_command)
    if len(caller_commands) > 0:
        print ("got " + str(len(caller_commands)) + " commands to run " + caller_name)
        for command in caller_commands:
            print("running command " + command)
            call(command,shell=True)
            print("Finished")
    else:
        print('Finished running ' + caller_name + ' commands')



Finished running Pindel commands
got 2 commands to run Lofreq
running command conda run -n kc_lofreq bash -c " rm -f /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.lofreq/lofreq* && lofreq indelqual --dindel -f genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa -o /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.lofreq/lofreq.lofreq_indelqual.sim.bam /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.bam && samtools index /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.lofreq/lofreq.lofreq_indelqual.sim.bam && lofreq indelqual --dindel -f genomes/Homo_sapiens/UCSC

/uufs/chpc.utah.edu/common/home/clementm-group1/conda/mambaforge/envs/kc_lofreq/bin/lofreq2_somatic.py:784: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  LOG.warn("No dbsnp file given. Using dbsnp is highly recommended"
WARNING [2025-07-22 18:10:20,517]: No dbsnp file given. Using dbsnp is highly recommended when dealing with human data.
/uufs/chpc.utah.edu/common/home/clementm-group1/conda/mambaforge/envs/kc_lofreq/bin/lofreq2_somatic.py:513: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  LOG.warn("uniq stderr: %s" % l.decode())
WARNING [2025-07-22 18:10:32,161]: uniq stderr: WARNING(lofreq_uniq.c|main_uniq): Didn't find any variants in input




Finished
running command conda run -n kc_lofreq bash -c " rm -f /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.lofreq/lofreq* && lofreq indelqual --dindel -f genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa -o /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.lofreq/lofreq.lofreq_indelqual.sim.bam /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.bam && samtools index /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.lofreq/lofreq.lofreq_indelqual.sim.bam && lofreq indelqual --dindel -f genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa -o /uufs/chpc.utah.

/uufs/chpc.utah.edu/common/home/clementm-group1/conda/mambaforge/envs/kc_lofreq/bin/lofreq2_somatic.py:784: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  LOG.warn("No dbsnp file given. Using dbsnp is highly recommended"
WARNING [2025-07-22 18:10:52,620]: No dbsnp file given. Using dbsnp is highly recommended when dealing with human data.
/uufs/chpc.utah.edu/common/home/clementm-group1/conda/mambaforge/envs/kc_lofreq/bin/lofreq2_somatic.py:513: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  LOG.warn("uniq stderr: %s" % l.decode())
WARNING [2025-07-22 18:11:09,226]: uniq stderr: WARNING(lofreq_uniq.c|main_uniq): Didn't find any variants in input

/uufs/chpc.utah.edu/common/home/clementm-group1/conda/mambaforge/envs/kc_lofreq/bin/lofreq2_somatic.py:513: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  LOG.warn("uniq stderr: %s" % l.decode())
WARNING [2025-07-22 18:11:09,231]: uniq stderr: WARNING(lofr

Finished
got 2 commands to run VarDict
running command conda run -n kc_vardict bash -c " vardict -f 0.0001 -G genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa -b '/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.bam|/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.ctl.bam'  -R chr11:835197-837397 -r 1 | testsomatic.R | var2vcf_paired.pl -A -N 'simulated|simulatedCtl' -f 0.0001 > /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.vardict/vardict.vcf && touch /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.vardict/vardict.finished"
Finished
running command conda run -n kc_

...

Using GATK jar /uufs/chpc.utah.edu/common/home/clementm-group1/conda/mambaforge/envs/kc_gatk4/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /uufs/chpc.utah.edu/common/home/clementm-group1/conda/mambaforge/envs/kc_gatk4/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar Mutect2 -R genomes/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa -I /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.bam -I /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.ctl.bam -normal simulatedCtl -O /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30


Successfully created workflow run script.
To execute the workflow, run the following script and set appropriate options:

/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.strelka/manta/runWorkflow.py

Successfully created workflow run script.
To execute the workflow, run the following script and set appropriate options:

/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.strelka/runWorkflow.py

Finished
running command conda run -n kc_strelka2 bash -c " rm -rf /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.strelka && rm -rf /uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0

[2025-07-23T00:12:16.817620Z] [notchpeak30.int.chpc.utah.edu] [2018175_1] [WorkflowRunner] Initiating pyFlow run
[2025-07-23T00:12:16.817807Z] [notchpeak30.int.chpc.utah.edu] [2018175_1] [WorkflowRunner] pyFlowClientWorkflowClass: MantaWorkflow
[2025-07-23T00:12:16.817821Z] [notchpeak30.int.chpc.utah.edu] [2018175_1] [WorkflowRunner] pyFlowVersion: 1.1.20
[2025-07-23T00:12:16.817831Z] [notchpeak30.int.chpc.utah.edu] [2018175_1] [WorkflowRunner] pythonVersion: 2.7.18.final.0
[2025-07-23T00:12:16.817839Z] [notchpeak30.int.chpc.utah.edu] [2018175_1] [WorkflowRunner] WorkingDir: '/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs'
[2025-07-23T00:12:16.817847Z] [notchpeak30.int.chpc.utah.edu] [2018175_1] [WorkflowRunner] ProcessCmdLine: '/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD1_r0.caller.strelka/manta/runWorkflow.py -m local -j 20'
[202


Successfully created workflow run script.
To execute the workflow, run the following script and set appropriate options:

/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.strelka/manta/runWorkflow.py

Successfully created workflow run script.
To execute the workflow, run the following script and set appropriate options:

/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.strelka/runWorkflow.py

Finished


[2025-07-23T00:12:56.822247Z] [notchpeak30.int.chpc.utah.edu] [2029488_1] [WorkflowRunner] Initiating pyFlow run
[2025-07-23T00:12:56.822401Z] [notchpeak30.int.chpc.utah.edu] [2029488_1] [WorkflowRunner] pyFlowClientWorkflowClass: MantaWorkflow
[2025-07-23T00:12:56.822416Z] [notchpeak30.int.chpc.utah.edu] [2029488_1] [WorkflowRunner] pyFlowVersion: 1.1.20
[2025-07-23T00:12:56.822425Z] [notchpeak30.int.chpc.utah.edu] [2029488_1] [WorkflowRunner] pythonVersion: 2.7.18.final.0
[2025-07-23T00:12:56.822433Z] [notchpeak30.int.chpc.utah.edu] [2029488_1] [WorkflowRunner] WorkingDir: '/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs'
[2025-07-23T00:12:56.822442Z] [notchpeak30.int.chpc.utah.edu] [2029488_1] [WorkflowRunner] ProcessCmdLine: '/uufs/chpc.utah.edu/common/home/clementm-group1/projects/20230508_uwgs/analysis/mutsuite/mutsuite/docs/simSettings.txt.output/sim_d30_p5.0_q0_sD2_r0.caller.strelka/manta/runWorkflow.py -m local -j 20'
[202

## Read in results from each caller

In [14]:
def print_mutations(called_mutations, simulated_mutations, callers, filename=None):
    """
    Prints the mutations found by each caller in a tab-separated format.
    If filename is provided, writes to that file instead of printing to stdout.
    """

    # Prepare header
    head_string = "BP\tMUTATION\tMUTINFO\tSIMULATED"
    caller_names = []
    all_mutations = {}  # not separated by caller
    for caller in callers:
        caller_name = caller.get_name()
        caller_names.append(caller_name)
        head_string += "\t"+caller_name
        for mutation in called_mutations[caller_name]:
            if mutation not in all_mutations:
                all_mutations[mutation] = 0
    
    sorted_mutations = sorted(all_mutations.keys(), key=lambda x: x.split()[0], reverse=False)
    outstring = ""
    for mutation in sorted_mutations:
        outstring += mutation.replace(" ", "\t")
        sim_count = 0
        if mutation in simulated_mutations:
            sim_count = simulated_mutations[mutation].sim_mut_count
        outstring += "\t" + str(sim_count)

        for caller_name in caller_names:
            caller_count = 0
            if caller_name in called_mutations and mutation in called_mutations[caller_name]:
                caller_count = called_mutations[caller_name][mutation]
            outstring += "\t" + str(caller_count)
        outstring += "\n"

    outstring = head_string + "\n" + outstring
    if filename:
        with open(filename, 'w') as f:
            f.write(outstring)
    else:
        print(outstring)
    return outstring

Now, for a specific example, we can aggregate all of the mutations found by each caller and print them out.

In [16]:
name, simulated_bam, simulated_control = samples[0]

mutations_by_caller = {} #contains information for each caller
all_mutations = {} # not sorted by caller
for caller in callers:
    caller_name = caller.get_name()
    mutations_by_caller[caller_name] = defaultdict(int)
    caller_passing_indels, caller_all_mutations = caller.get_results(simulated_bam,simulated_control)
    for mutation_name, mutation in caller_all_mutations.items():
        mut_count = mutation.sim_mut_count
        mutations_by_caller[caller_name][mutation_name] = mut_count
        all_mutations[mutation_name] = 1
print_mutations(called_mutations=mutations_by_caller,
    simulated_mutations=simulated_mutations[name], callers=callers)


BP	MUTATION	MUTINFO	SIMULATED	Pindel	Lofreq	VarDict	SomaticSniper	Varscan	Mutect	Strelka
831808	S	G	0	0	0	0	0	18	0	0
832902	D	5	0	0	0	0	0	12	0	0
832981	D	3	0	0	0	0	0	15	0	0
833261	S	G	0	0	0	0	0	13	0	0
833326	S	G	0	0	0	0	0	16	0	0
833628	S	G	0	0	0	0	0	20	0	0
833825	S	T	0	0	0	0	0	13	0	0
835131	S	C	0	0	0	0	0	27	0	0
835362	S	C	0	0	0	1	0	0	0	0
835438	S	C	0	0	0	3	0	0	0	2
835476	S	T	0	0	0	0	0	0	0	3
835662	S	A	0	0	0	1	0	0	0	0
835718	D	2	0	2	0	0	0	0	0	0
835718	S	G	0	0	0	14	0	11	0	0
835732	S	C	0	0	0	2	0	0	0	0
835771	S	C	0	0	0	21	0	18	0	0
836022	S	T	0	0	0	18	0	17	0	0
836180	S	T	0	0	0	1	0	0	0	0
836226	S	G	0	0	0	12	0	12	0	0
836295	D	1	5	5	5	5	0	0	5	5
836367	S	A	0	0	0	1	0	0	0	0
836538	D	1	0	20	0	10	0	10	0	0
836723	S	T	0	0	0	1	0	0	0	0
836753	S	C	0	0	0	1	0	0	0	0
836757	D	1	0	2	0	1	0	0	0	0
837072	D	1	0	2	0	1	0	0	0	0
837083	S	A	0	0	0	13	0	8	0	0
837224	S	A	0	0	0	1	0	0	0	0
837248	S	T	0	0	0	1	0	0	0	0
837365	S	C	0	0	0	1	0	0	0	0
838633	S	G	0	0	0	0	0	19	0	0
840180	S	T	0	0	0	0	0	13	0	0
840250	S	G	0	0	0	0	0	17	0

'BP\tMUTATION\tMUTINFO\tSIMULATED\tPindel\tLofreq\tVarDict\tSomaticSniper\tVarscan\tMutect\tStrelka\n831808\tS\tG\t0\t0\t0\t0\t0\t18\t0\t0\n832902\tD\t5\t0\t0\t0\t0\t0\t12\t0\t0\n832981\tD\t3\t0\t0\t0\t0\t0\t15\t0\t0\n833261\tS\tG\t0\t0\t0\t0\t0\t13\t0\t0\n833326\tS\tG\t0\t0\t0\t0\t0\t16\t0\t0\n833628\tS\tG\t0\t0\t0\t0\t0\t20\t0\t0\n833825\tS\tT\t0\t0\t0\t0\t0\t13\t0\t0\n835131\tS\tC\t0\t0\t0\t0\t0\t27\t0\t0\n835362\tS\tC\t0\t0\t0\t1\t0\t0\t0\t0\n835438\tS\tC\t0\t0\t0\t3\t0\t0\t0\t2\n835476\tS\tT\t0\t0\t0\t0\t0\t0\t0\t3\n835662\tS\tA\t0\t0\t0\t1\t0\t0\t0\t0\n835718\tD\t2\t0\t2\t0\t0\t0\t0\t0\t0\n835718\tS\tG\t0\t0\t0\t14\t0\t11\t0\t0\n835732\tS\tC\t0\t0\t0\t2\t0\t0\t0\t0\n835771\tS\tC\t0\t0\t0\t21\t0\t18\t0\t0\n836022\tS\tT\t0\t0\t0\t18\t0\t17\t0\t0\n836180\tS\tT\t0\t0\t0\t1\t0\t0\t0\t0\n836226\tS\tG\t0\t0\t0\t12\t0\t12\t0\t0\n836295\tD\t1\t5\t5\t5\t5\t0\t0\t5\t5\n836367\tS\tA\t0\t0\t0\t1\t0\t0\t0\t0\n836538\tD\t1\t0\t20\t0\t10\t0\t10\t0\t0\n836723\tS\tT\t0\t0\t0\t1\t0\t0\t0\t0\n836753